# 1. Build your own convolutional neural network using pytorch

In [6]:
!pip install torch torchvision

Defaulting to user installation because normal site-packages is not writeable


In [20]:
import os
import glob
import pandas as pd
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, Dataset
from torchvision import datasets, transforms
from PIL import Image
from sklearn.metrics import accuracy_score

# Define transformations for the training, validation, and testing data
train_transforms = transforms.Compose([
    transforms.Resize((75, 75)),  # Resize images to 75x75
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])

test_transforms = transforms.Compose([
    transforms.Resize((75, 75)),  # Resize images to 75x75
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])

# Load the datasets with ImageFolder
train_dir = r'C:\Users\srikr\Downloads\Dog_X_ray\Dog_X_ray\Dog_heart\Dog_heart\Train'
valid_dir = r'C:\Users\srikr\Downloads\Dog_X_ray\Dog_X_ray\Dog_heart\Dog_heart\Valid'
test_dir = r'C:\Users\srikr\Downloads\Dog_X_ray\Dog_X_ray\Test\Test'

train_data = datasets.ImageFolder(train_dir, transform=train_transforms)
valid_data = datasets.ImageFolder(valid_dir, transform=test_transforms)
train_loader = DataLoader(train_data, batch_size=32, shuffle=True)
valid_loader = DataLoader(valid_data, batch_size=32, shuffle=False)

# Custom dataset class for test data
class CustomTestDataset(Dataset):
    def __init__(self, image_paths, transform=None):
        self.image_paths = image_paths
        self.transform = transform

    def __len__(self):
        return len(self.image_paths)

    def __getitem__(self, idx):
        image_path = self.image_paths[idx]
        image = Image.open(image_path).convert('RGB')
        if self.transform:
            image = self.transform(image)
        return image, image_path

# Load test images
test_image_paths = glob.glob(os.path.join(test_dir, '*.png'))
test_data = CustomTestDataset(test_image_paths, transform=test_transforms)
test_loader = DataLoader(test_data, batch_size=32, shuffle=False)

# Define the CNN model
class SimpleCNN(nn.Module):
    def __init__(self, num_classes=2):
        super(SimpleCNN, self).__init__()
        self.features = nn.Sequential(
            nn.Conv2d(3, 64, kernel_size=3, padding=1),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(kernel_size=2, stride=2),
            nn.Conv2d(64, 128, kernel_size=3, padding=1),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(kernel_size=2, stride=2),
            nn.Conv2d(128, 256, kernel_size=3, padding=1),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(kernel_size=2, stride=2)
        )
        self.classifier = nn.Sequential(
            nn.Dropout(),
            nn.Linear(256 * 9 * 9, 1024),  # 75/8 = 9 (floor division)
            nn.ReLU(inplace=True),
            nn.Dropout(),
            nn.Linear(1024, num_classes)
        )

    def forward(self, x):
        x = self.features(x)
        x = x.view(x.size(0), -1)
        x = self.classifier(x)
        return x

# Initialize the model, criterion, and optimizer
model = SimpleCNN(num_classes=len(train_data.classes))
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

# Function to train the model
def train_model(model, criterion, optimizer, train_loader, valid_loader, num_epochs=2):
    for epoch in range(num_epochs):
        model.train()
        running_loss = 0.0
        for inputs, labels in train_loader:
            if torch.cuda.is_available():
                inputs, labels = inputs.cuda(), labels.cuda()

            optimizer.zero_grad()
            outputs = model(inputs)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()
            running_loss += loss.item() * inputs.size(0)

        epoch_loss = running_loss / len(train_loader.dataset)
        val_accuracy = validate_model(model, valid_loader)
        print(f'Epoch {epoch}/{num_epochs - 1}, Loss: {epoch_loss:.4f}, Validation Accuracy: {val_accuracy:.4f}')

# Function to validate the model
def validate_model(model, valid_loader):
    model.eval()
    all_preds = []
    all_labels = []
    with torch.no_grad():
        for inputs, labels in valid_loader:
            if torch.cuda.is_available():
                inputs, labels = inputs.cuda(), labels.cuda()
            outputs = model(inputs)
            _, preds = torch.max(outputs, 1)
            all_preds.extend(preds.cpu().numpy())
            all_labels.extend(labels.cpu().numpy())
    return accuracy_score(all_labels, all_preds)

# Function to evaluate the model
def evaluate_model(model, test_loader):
    model.eval()
    predictions = []
    image_paths = []
    with torch.no_grad():
        for inputs, paths in test_loader:
            if torch.cuda.is_available():
                inputs = inputs.cuda()
            outputs = model(inputs)
            _, preds = torch.max(outputs, 1)
            predictions.extend(preds.cpu().numpy())
            image_paths.extend(paths)
    return predictions, image_paths

# Training the model
if torch.cuda.is_available():
    model = model.cuda()
train_model(model, criterion, optimizer, train_loader, valid_loader, num_epochs=2)

# Save the model weights
torch.save(model.state_dict(), 'model_weights.pth')

# Evaluating the model
predictions, image_paths = evaluate_model(model, test_loader)

# Save the results
results = pd.DataFrame({'Image': image_paths, 'Prediction': predictions})
results.to_csv('cnn_predictions.csv', index=False, header=False)


Epoch 0/1, Loss: 1.1440, Validation Accuracy: 0.5400
Epoch 1/1, Loss: 0.9069, Validation Accuracy: 0.5750


In [3]:
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import datasets, transforms, models
from torch.utils.data import DataLoader, Dataset
from PIL import Image
import os

# Define paths to the datasets
train_dir = r'C:\Users\srikr\Downloads\Dog_X_ray\Dog_X_ray\Dog_heart\Dog_heart\Train'
valid_dir = r'C:\Users\srikr\Downloads\Dog_X_ray\Dog_X_ray\Dog_heart\Dog_heart\Valid'
test_dir = r'C:\Users\srikr\Downloads\Dog_X_ray\Dog_X_ray\Test'

# Define transformations for the training, validation, and test sets
transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])

# Load the training and validation datasets using ImageFolder
train_dataset = datasets.ImageFolder(root=train_dir, transform=transform)
valid_dataset = datasets.ImageFolder(root=valid_dir, transform=transform)

# Define the DataLoaders
train_loader = DataLoader(dataset=train_dataset, batch_size=32, shuffle=True)
valid_loader = DataLoader(dataset=valid_dataset, batch_size=32, shuffle=False)

# Define a custom dataset for the test set to return image tensors and file names
class CustomTestDataset(Dataset):
    def __init__(self, root_dir, transform=None):
        self.root_dir = root_dir
        self.transform = transform
        self.image_paths = [os.path.join(root_dir, fname) for fname in os.listdir(root_dir) if fname.endswith('.jpg')]

    def __len__(self):
        return len(self.image_paths)

    def __getitem__(self, idx):
        img_path = self.image_paths[idx]
        image = Image.open(img_path).convert('RGB')
        if self.transform:
            image = self.transform(image)
        return image, os.path.basename(img_path)

# Load the test dataset using the custom dataset class
test_dataset = CustomTestDataset(root_dir=test_dir, transform=transform)
test_loader = DataLoader(dataset=test_dataset, batch_size=32, shuffle=False)

# Define the CNN model
class CustomCNN(nn.Module):
    def __init__(self, num_classes=3):
        super(CustomCNN, self).__init__()
        
        # Feature extraction layers
        self.features = nn.Sequential(
            nn.Conv2d(3, 64, kernel_size=7, stride=2, padding=3),  # Example: AlexNet
            nn.BatchNorm2d(64),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(kernel_size=3, stride=2, padding=1),
            
            nn.Conv2d(64, 128, kernel_size=3, padding=1),
            nn.BatchNorm2d(128),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(kernel_size=2, stride=2),
            
            nn.Conv2d(128, 256, kernel_size=3, padding=1),
            nn.BatchNorm2d(256),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(kernel_size=2, stride=2),
            
            nn.Conv2d(256, 512, kernel_size=3, padding=1),
            nn.BatchNorm2d(512),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(kernel_size=2, stride=2),
        )
        
        # Fully connected layers
        self.classifier = nn.Sequential(
            nn.Dropout(),
            nn.Linear(512 * 7 * 7, 4096),  # Example: VGG-16
            nn.ReLU(inplace=True),
            nn.Dropout(),
            nn.Linear(4096, 4096),
            nn.ReLU(inplace=True),
            nn.Linear(4096, num_classes)
        )
        
    def forward(self, x):
        x = self.features(x)
        x = x.view(x.size(0), -1)
        x = self.classifier(x)
        return x

# Initialize the model, loss function, and optimizer
model = CustomCNN(num_classes=3)
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

# Training the model
num_epochs = 10
best_val_loss = float('inf')

for epoch in range(num_epochs):
    model.train()
    train_loss = 0.0
    for images, labels in train_loader:
        outputs = model(images)
        loss = criterion(outputs, labels)
        
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        
        train_loss += loss.item() * images.size(0)
        
    train_loss /= len(train_loader.dataset)
    
    model.eval()
    val_loss = 0.0
    with torch.no_grad():
        for images, labels in valid_loader:
            outputs = model(images)
            loss = criterion(outputs, labels)
            val_loss += loss.item() * images.size(0)
            
    val_loss /= len(valid_loader.dataset)
    
    print(f'Epoch [{epoch+1}/{num_epochs}], Train Loss: {train_loss:.4f}, Val Loss: {val_loss:.4f}')
    
    # Save the best model
    if val_loss < best_val_loss:
        best_val_loss = val_loss
        torch.save(model.state_dict(), 'best_model.pth')

# Load the best model for evaluation
model.load_state_dict(torch.load('best_model.pth'))

# Evaluation on the test set
model.eval()
test_loss = 0.0
correct = 0
total = 0
with torch.no_grad():
    for images, file_names in test_loader:
        outputs = model(images)
        # You can use outputs and file_names for your custom evaluation logic
        # For now, we'll assume that we don't have labels for the test set
        loss = criterion(outputs, labels)
        test_loss += loss.item() * images.size(0)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()



Epoch [1/10], Train Loss: 7.8100, Val Loss: 1.0512
Epoch [2/10], Train Loss: 0.9999, Val Loss: 0.9821
Epoch [3/10], Train Loss: 0.9419, Val Loss: 0.9183
Epoch [4/10], Train Loss: 0.9069, Val Loss: 0.8980
Epoch [5/10], Train Loss: 0.8779, Val Loss: 0.9099
Epoch [6/10], Train Loss: 0.8592, Val Loss: 0.9090
Epoch [7/10], Train Loss: 0.8468, Val Loss: 0.8683
Epoch [8/10], Train Loss: 0.8312, Val Loss: 0.8952
Epoch [9/10], Train Loss: 0.8384, Val Loss: 0.8092
Epoch [10/10], Train Loss: 0.8253, Val Loss: 0.7923


ZeroDivisionError: float division by zero

In [ ]:
# Assuming you have labels, uncomment and update the lines above to compute accuracy
test_loss /= len(test_loader.dataset)
accuracy = 100 * correct / total
print(f'Test Loss: {test_loss:.4f}, Test Accuracy: {accuracy:.2f}%')

In [4]:
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import datasets, transforms, models
from torch.utils.data import DataLoader, Dataset
from PIL import Image
import os
import pandas as pd

# Define paths to the datasets
train_dir = r'C:\Users\srikr\Downloads\Dog_X_ray\Dog_X_ray\Dog_heart\Dog_heart\Train'
valid_dir = r'C:\Users\srikr\Downloads\Dog_X_ray\Dog_X_ray\Dog_heart\Dog_heart\Valid'
test_dir = r'C:\Users\srikr\Downloads\Dog_X_ray\Dog_X_ray\Test'

# Define transformations for the training, validation, and test sets
transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])

# Load the training and validation datasets using ImageFolder
train_dataset = datasets.ImageFolder(root=train_dir, transform=transform)
valid_dataset = datasets.ImageFolder(root=valid_dir, transform=transform)

# Define the DataLoaders
train_loader = DataLoader(dataset=train_dataset, batch_size=32, shuffle=True)
valid_loader = DataLoader(dataset=valid_dataset, batch_size=32, shuffle=False)

# Define a custom dataset for the test set to return image tensors and file names
class CustomTestDataset(Dataset):
    def __init__(self, root_dir, transform=None):
        self.root_dir = root_dir
        self.transform = transform
        self.image_paths = [os.path.join(root_dir, fname) for fname in os.listdir(root_dir) if fname.endswith('.jpg')]

    def __len__(self):
        return len(self.image_paths)

    def __getitem__(self, idx):
        img_path = self.image_paths[idx]
        image = Image.open(img_path).convert('RGB')
        if self.transform:
            image = self.transform(image)
        return image, os.path.basename(img_path)

# Load the test dataset using the custom dataset class
test_dataset = CustomTestDataset(root_dir=test_dir, transform=transform)
test_loader = DataLoader(dataset=test_dataset, batch_size=32, shuffle=False)

# Define the CNN model
class CustomCNN(nn.Module):
    def __init__(self, num_classes=3):
        super(CustomCNN, self).__init__()
        
        # Feature extraction layers
        self.features = nn.Sequential(
            nn.Conv2d(3, 64, kernel_size=7, stride=2, padding=3),  # Example: AlexNet
            nn.BatchNorm2d(64),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(kernel_size=3, stride=2, padding=1),
            
            nn.Conv2d(64, 128, kernel_size=3, padding=1),
            nn.BatchNorm2d(128),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(kernel_size=2, stride=2),
            
            nn.Conv2d(128, 256, kernel_size=3, padding=1),
            nn.BatchNorm2d(256),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(kernel_size=2, stride=2),
            
            nn.Conv2d(256, 512, kernel_size=3, padding=1),
            nn.BatchNorm2d(512),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(kernel_size=2, stride=2),
        )
        
        # Fully connected layers
        self.classifier = nn.Sequential(
            nn.Dropout(),
            nn.Linear(512 * 7 * 7, 4096),  # Example: VGG-16
            nn.ReLU(inplace=True),
            nn.Dropout(),
            nn.Linear(4096, 4096),
            nn.ReLU(inplace=True),
            nn.Linear(4096, num_classes)
        )
        
    def forward(self, x):
        x = self.features(x)
        x = x.view(x.size(0), -1)
        x = self.classifier(x)
        return x

# Initialize the model, loss function, and optimizer
model = CustomCNN(num_classes=3)
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

# Training the model
num_epochs = 10
best_val_loss = float('inf')

for epoch in range(num_epochs):
    model.train()
    train_loss = 0.0
    for images, labels in train_loader:
        outputs = model(images)
        loss = criterion(outputs, labels)
        
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        
        train_loss += loss.item() * images.size(0)
        
    train_loss /= len(train_loader.dataset)
    
    model.eval()
    val_loss = 0.0
    with torch.no_grad():
        for images, labels in valid_loader:
            outputs = model(images)
            loss = criterion(outputs, labels)
            val_loss += loss.item() * images.size(0)
            
    val_loss /= len(valid_loader.dataset)
    
    print(f'Epoch [{epoch+1}/{num_epochs}], Train Loss: {train_loss:.4f}, Val Loss: {val_loss:.4f}')
    
    # Save the best model
    if val_loss < best_val_loss:
        best_val_loss = val_loss
        torch.save(model.state_dict(), 'best_model.pth')

# Load the best model for evaluation
model.load_state_dict(torch.load('best_model.pth'))

# Evaluation on the test set
model.eval()
predictions = []
file_names = []

with torch.no_grad():
    for images, fnames in test_loader:
        outputs = model(images)
        probabilities = torch.softmax(outputs, dim=1)
        _, predicted = torch.max(probabilities, 1)
        predictions.extend(predicted.cpu().numpy())
        file_names.extend(fnames)

# Convert array to DataFrame with array name as column name
df_nn = pd.DataFrame({'filenames': file_names, 'nn_predictions': predictions})

df_nn.to_csv('pytorch_pred.csv', index=False, header=True)

print(df_nn.head())

Epoch [1/10], Train Loss: 5.8180, Val Loss: 0.9905
Epoch [2/10], Train Loss: 0.9783, Val Loss: 0.9349
Epoch [3/10], Train Loss: 0.9308, Val Loss: 0.8720
Epoch [4/10], Train Loss: 0.8944, Val Loss: 0.8535
Epoch [5/10], Train Loss: 0.8535, Val Loss: 0.8295
Epoch [6/10], Train Loss: 0.8289, Val Loss: 0.8136
Epoch [7/10], Train Loss: 0.8334, Val Loss: 0.7692
Epoch [8/10], Train Loss: 0.7876, Val Loss: 0.7894
Epoch [9/10], Train Loss: 0.7754, Val Loss: 0.8258
Epoch [10/10], Train Loss: 0.7787, Val Loss: 0.8381
Empty DataFrame
Columns: [filenames, nn_predictions]
Index: []


In [5]:
# Convert array to DataFrame with array name as column name
df_nn = pd.DataFrame({'filenames': file_names, 'nn_predictions': predictions})

df_nn.to_csv('pytorch_pred.csv', index=False, header=True)

print(df_nn.head())

Empty DataFrame
Columns: [filenames, nn_predictions]
Index: []


### Overview of Convolutional Neural Networks (CNNs)

1. **Convolutional Layers**:
    - **Filters/Kernels**: Small-sized matrices used for convolution operations to extract features from the input image.
    - **Stride**: The number of pixels by which we slide the filter matrix over the input matrix.
    - **Padding**: Adding zeros around the input matrix to maintain the spatial dimensions after convolution.

2. **Pooling Layers**:
    - **Max Pooling**: Selects the maximum value from the region covered by the filter.
    - **Average Pooling**: Computes the average of all values from the region covered by the filter.

3. **Fully Connected (FC) Layers**:
    - Dense layers where every neuron is connected to every neuron in the previous layer.
    - Often followed by an activation function like softmax for classification tasks.

### CNN Architecture for Dog Heart Dataset

Here's a sample architecture for a CNN to classify images in the dog heart dataset:

1. **Input Layer**: Images of size 75x75x3 (3 channels for RGB).
2. **Convolutional Layer 1**: 32 filters, kernel size 3x3, stride 1, padding 1.
3. **ReLU Activation**
4. **Max Pooling Layer 1**: Pool size 2x2, stride 2.
5. **Convolutional Layer 2**: 64 filters, kernel size 3x3, stride 1, padding 1.
6. **ReLU Activation**
7. **Max Pooling Layer 2**: Pool size 2x2, stride 2.
8. **Flatten Layer**: Convert the 2D matrix to a 1D vector.
9. **Fully Connected Layer 1**: 128 neurons.
10. **ReLU Activation**
11. **Output Layer**: Softmax activation for classification.

### Key Points from the PDFs
- **Lecture 5** covers CNN operations, classical CNN models like LeNet-5, AlexNet, VGG-16, ResNet, and DenseNet【73†source】.
- **Parameters in CNN** section explains how to calculate trainable parameters in each layer【73†source】.
- **Visualization and Application in Veterinary Science** highlights the use of CNNs in medical and veterinary applications, which is relevant for the dog heart dataset project【73†source】.

This code provides a basic framework, which can be further refined and optimized based on the specific needs and dataset characteristics.

In [22]:
### Implementing the CNN in PyTorch

import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import datasets, transforms
from torch.utils.data import DataLoader

# Define the CNN architecture
class DogHeartCNN(nn.Module):
    def __init__(self):
        super(DogHeartCNN, self).__init__()
        self.conv1 = nn.Conv2d(in_channels=3, out_channels=32, kernel_size=3, stride=1, padding=1)
        self.pool = nn.MaxPool2d(kernel_size=2, stride=2, padding=0)
        self.conv2 = nn.Conv2d(in_channels=32, out_channels=64, kernel_size=3, stride=1, padding=1)
        self.fc1 = nn.Linear(64 * 18 * 18, 128)  # Adjust input size according to pooling
        self.fc2 = nn.Linear(128, 2)  # Assuming 2 classes for dog heart conditions

    def forward(self, x):
        x = self.pool(F.relu(self.conv1(x)))
        x = self.pool(F.relu(self.conv2(x)))
        x = x.view(-1, 64 * 18 * 18)  # Flatten the tensor
        x = F.relu(self.fc1(x))
        x = self.fc2(x)
        return x

# Define the data transformations
transform = transforms.Compose([
    transforms.Resize((75, 75)),
    transforms.ToTensor()
])

# Load the dataset
train_data = datasets.ImageFolder(root=r'C:\Users\srikr\Downloads\Dog_X_ray\Dog_X_ray\Dog_heart\Dog_heart\Train', transform=transform)
val_data = datasets.ImageFolder(root=r'C:\Users\srikr\Downloads\Dog_X_ray\Dog_X_ray\Dog_heart\Dog_heart\Valid', transform=transform)
test_data = datasets.ImageFolder(root=r'C:\Users\srikr\Downloads\Dog_X_ray\Dog_X_ray\Test\Test', transform=transform)

train_loader = DataLoader(train_data, batch_size=32, shuffle=True)
val_loader = DataLoader(val_data, batch_size=32, shuffle=True)
test_loader = DataLoader(test_data, batch_size=32, shuffle=False)

# Initialize the model, loss function and optimizer
model = DogHeartCNN()
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

# Training the model
num_epochs = 20
for epoch in range(num_epochs):
    model.train()
    running_loss = 0.0
    for inputs, labels in train_loader:
        optimizer.zero_grad()
        outputs = model(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        running_loss += loss.item()
    print(f"Epoch {epoch+1}, Loss: {running_loss/len(train_loader)}")

# Validation and Testing can be done similarly

FileNotFoundError: Couldn't find any class folder in C:\Users\srikr\Downloads\Dog_X_ray\Dog_X_ray\Test\Test.

In [24]:
import torch
from torchvision import transforms, datasets

# Define data transformations
transform = transforms.Compose([
    transforms.Resize((128, 128)),  # Resize images to 128x128
    transforms.ToTensor(),          # Convert images to tensors
    transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))  # Normalize images
])

# Load the training dataset
train_dataset = datasets.ImageFolder(root=r'C:\Users\srikr\Downloads\Dog_X_ray\Dog_X_ray\Dog_heart\Dog_heart\Train', transform=transform)
train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=64, shuffle=True)

# Custom DataLoader for test dataset
class CustomDataset(torch.utils.data.Dataset):
    def __init__(self, file_list, transform=None):
        self.file_list = file_list
        self.transform = transform

    def __len__(self):
        return len(self.file_list)

    def __getitem__(self, index):
        img_path = self.file_list[index]
        img = Image.open(img_path)
        if self.transform:
            img = self.transform(img)
        return img, img_path

# Example usage:
test_file_list = ['path_to_test_image1.jpg', 'path_to_test_image2.jpg', ...]
test_dataset = CustomDataset(test_file_list, transform=transform)
test_loader = torch.utils.data.DataLoader(test_dataset, batch_size=1, shuffle=False)


In [25]:
import torch.nn as nn
import torch.nn.functional as F

class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.conv1 = nn.Conv2d(3, 32, 3, stride=1, padding=1)
        self.pool = nn.MaxPool2d(2, 2)
        self.conv2 = nn.Conv2d(32, 64, 3, stride=1, padding=1)
        self.conv3 = nn.Conv2d(64, 128, 3, stride=1, padding=1)
        self.fc1 = nn.Linear(128 * 16 * 16, 128)
        self.fc2 = nn.Linear(128, 1)

    def forward(self, x):
        x = self.pool(F.relu(self.conv1(x)))
        x = self.pool(F.relu(self.conv2(x)))
        x = self.pool(F.relu(self.conv3(x)))
        x = x.view(-1, 128 * 16 * 16)
        x = F.relu(self.fc1(x))
        x = torch.sigmoid(self.fc2(x))
        return x

# Initialize the model
model = Net()

In [26]:
import torch.optim as optim

criterion = nn.BCELoss()
optimizer = optim.SGD(model.parameters(), lr=0.001, momentum=0.9)

# Training loop
epochs = 20
for epoch in range(epochs):
    running_loss = 0.0
    for i, data in enumerate(train_loader, 0):
        inputs, labels = data
        optimizer.zero_grad()

        # Forward + backward + optimize
        outputs = model(inputs)
        loss = criterion(outputs, labels.unsqueeze(1).float())
        loss.backward()
        optimizer.step()

        running_loss += loss.item()

        # Print statistics every 2000 batches
        if i % 2000 == 1999:
            print('[%d, %5d] loss: %.3f' %
                  (epoch + 1, i + 1, running_loss / 2000))
            running_loss = 0.0

print('Finished Training')

# Evaluation
correct = 0
total = 0
with torch.no_grad():
    for data in test_loader:
        images, labels = data
        outputs = model(images)
        predicted = torch.round(outputs)
        total += labels.size(0)
        correct += (predicted == labels.unsqueeze(1)).sum().item()

print('Accuracy of the network on the test images: %d %%' % (
    100 * correct / total))


Finished Training


FileNotFoundError: [Errno 2] No such file or directory: 'path_to_test_image1.jpg'

# 2. Train your model using dog heart dataset (you may need to use  Google Colab (or Kaggle) with GPU to train your code) 

### (1) use torchvision.datasets.ImageFolder for the training dataset
### (2) use custom dataloader for test dataset (return image tensor and file name)

# 3. Evaluate your model using the developed software

# 4. Compare results with [RVT paper](https://www.nature.com/articles/s41598-023-50063-x). Requirement: performance is better than VGG16: 70%

# 5. Write a four-page paper report using the shared LaTex template. Upload your paper to ResearchGate or Arxiv, and put your paper link and GitHub weight link here.

# 6. Grading rubric

(1). Code ------- 20 points (you also need to upload your final model as a pt file)

(2). Grammer ---- 20 points

(3). Introduction & related work --- 10 points


(4). Method  ---- 20 points

(5). Results ---- 20 points

     > = 70 % -->10 points
     < 50 % -->0 points
     >= 50 % & < 70% --> 0.5 point/percent
     

(6). Discussion - 10 points